In [4]:
!python3 --version

Python 3.10.12


In [1]:
import pyspark
from pyspark.sql import SparkSession
import os


## DEFINE SENSITIVE VARIABLES
NESSIE_URI = os.environ.get("NESSIE_URI") ## Nessie Server URI
WAREHOUSE = os.environ.get("WAREHOUSE") ## BUCKET TO WRITE DATA TOO
WAREHOUSE_BRONZE = os.environ.get("WAREHOUSE_BRONZE") ## BUCKET TO WRITE DATA TOO
WAREHOUSE_SYLVER = os.environ.get("WAREHOUSE_SYLVER") ## BUCKET TO WRITE DATA TOO
WAREHOUSE_GOLD = os.environ.get("WAREHOUSE_GOLD") ## BUCKET TO WRITE DATA TOO
AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID") ## AWS CREDENTIALS
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY") ## AWS CREDENTIALS
AWS_S3_ENDPOINT= os.environ.get("AWS_S3_ENDPOINT") ## MINIO ENDPOINT


print(AWS_S3_ENDPOINT)
print(NESSIE_URI)
print(WAREHOUSE)
print(AWS_ACCESS_KEY_ID)
print(AWS_SECRET_ACCESS_KEY)
print(WAREHOUSE_SYLVER)

http://minio:9000
http://nessie:19120/api/v1
s3a://warehouse/
M1s2Sa2IecSYl6SR6n4W
Kj6gSYsC1Q5MJ6VXWVW1S1m8eC8gPKxtLcOxB2wk
s3a://sylver-warehouse/


In [9]:
# AWS_REGION is used by Spark
AWS_REGION="us-east-1"
# This must match if using minio
MINIO_REGION="us-east-1"
# Used by pyIceberg
AWS_DEFAULT_REGION="us-east-1"
# AWS Credentials (this can use minio credential, to be filled in later)
AWS_ACCESS_KEY_ID="YRUl5sf7KF7WTVAAgTSO"
AWS_SECRET_ACCESS_KEY="gWIjs4ljGZJHcum9RlZtKTrL5La7FNWXZvwppm6L"
# If using Minio, this should be the API address of Minio Server
AWS_S3_ENDPOINT="http://minio:9000"
# Location where files will be written when creating new tables
WAREHOUSE="s3a://warehouse/"
# URI of Nessie Catalog
NESSIE_URI="http://nessie:19120/api/v1"

In [2]:
#### not use
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.102.5,software.amazon.awssdk:bundle:2.20.131,software.amazon.awssdk:url-connection-client:2.20.131')
        .set('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.bronze', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.bronze.uri', NESSIE_URI)
        .set('spark.sql.catalog.bronze.ref', 'main')
        .set('spark.sql.catalog.bronze.authentication.type', 'NONE')  # ✅ Move auth to "bronze" catalog
        .set('spark.sql.catalog.bronze.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')  # ✅ Use "bronze" prefix
        .set('spark.sql.catalog.bronze.s3.path-style-access', 'true')  # ✅ Configure S3 for "bronze"
        .set('spark.sql.catalog.bronze.s3.endpoint', AWS_S3_ENDPOINT)  # ✅ Fixed typo (no 's' at end)
        .set('spark.sql.catalog.bronze.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.bronze.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')  # ✅ Use "bronze" prefix
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY_ID)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.path.style.access','true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
)


In [25]:
## base config
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.102.5,software.amazon.awssdk:bundle:2.20.131,software.amazon.awssdk:url-connection-client:2.20.131')
        .set('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY_ID)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.path.style.access','true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
)

In [27]:
## config bronze

conf.set('spark.sql.catalog.bronze', 'org.apache.iceberg.spark.SparkCatalog')\
.set('spark.sql.catalog.bronze.uri', NESSIE_URI)\
.set('spark.sql.catalog.bronze.ref', 'main')\
.set('spark.sql.catalog.bronze.authentication.type', 'NONE')\
.set('spark.sql.catalog.bronze.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')\
.set('spark.sql.catalog.bronze.s3.path-style-access', 'true')\
.set('spark.sql.catalog.bronze.s3.endpoint', AWS_S3_ENDPOINT)\
.set('spark.sql.catalog.bronze.warehouse', WAREHOUSE_BRONZE)


In [14]:
## Sylver settings catalog

conf.set('spark.sql.catalog.sylver', 'org.apache.iceberg.spark.SparkCatalog')\
    .set('spark.sql.catalog.sylver.uri',NESSIE_URI)\
    .set('spark.sql.catalog.sylver.ref', 'main')\
    .set('spark.sql.catalog.sylver.authentication.type', 'NONE')\
    .set('spark.sql.catalog.sylver.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')\
    .set('spark.sql.catalog.sylver.s3.path-style-access', 'true')\
    .set('spark.sql.catalog.sylver.s3.endpoint',AWS_S3_ENDPOINT)\
    .set('spark.sql.catalog.sylver.warehouse',WAREHOUSE_BRONZE)\
    .set('spark.sql.catalog.sylver.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')\



In [50]:
## Bronze settings catalog

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.102.5,software.amazon.awssdk:bundle:2.20.131,software.amazon.awssdk:url-connection-client:2.20.131')
        .set('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.sylver', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.sylver.uri',NESSIE_URI)
        .set('spark.sql.catalog.sylver.ref', 'main')
        .set('spark.sql.catalog.sylver.authentication.type', 'NONE')  # ✅ Move auth to "bronze" catalog
        .set('spark.sql.catalog.sylver.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')  # ✅ Use "bronze" prefix
        .set('spark.sql.catalog.sylver.s3.path-style-access', 'true')  # ✅ Configure S3 for "bronze"
        .set('spark.sql.catalog.sylver.s3.endpoint',AWS_S3_ENDPOINT)  # ✅ Fixed typo (no 's' at end)
        .set('spark.sql.catalog.sylver.warehouse',WAREHOUSE_BRONZE)
        .set('spark.sql.catalog.sylver.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')  # ✅ Use "bronze" prefix
        .set('spark.hadoop.fs.s3a.access.key',AWS_ACCESS_KEY_ID)
        .set('spark.hadoop.fs.s3a.secret.key',AWS_SECRET_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.path.style.access','true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
)


In [50]:
## Sylver settings catalog

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.102.5,software.amazon.awssdk:bundle:2.20.131,software.amazon.awssdk:url-connection-client:2.20.131')
        .set('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.sylver', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.sylver.uri', NESSIE_URI)
        .set('spark.sql.catalog.sylver.ref', 'main')
        .set('spark.sql.catalog.sylver.authentication.type', 'NONE')  # ✅ Move auth to "bronze" catalog
        .set('spark.sql.catalog.sylver.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')  # ✅ Use "bronze" prefix
        .set('spark.sql.catalog.sylver.s3.path-style-access', 'true')  # ✅ Configure S3 for "bronze"
        .set('spark.sql.catalog.sylver.s3.endpoint', AWS_S3_ENDPOINT)  # ✅ Fixed typo (no 's' at end)
        .set('spark.sql.catalog.sylver.warehouse', WAREHOUSE_SYLVER)
        .set('spark.sql.catalog.sylver.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')  # ✅ Use "bronze" prefix
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY_ID)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.path.style.access','true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
)


In [50]:
## Gold settings catalog

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.102.5,software.amazon.awssdk:bundle:2.20.131,software.amazon.awssdk:url-connection-client:2.20.131')
        .set('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.sylver', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.sylver.uri', NESSIE_URI)
        .set('spark.sql.catalog.sylver.ref', 'main')
        .set('spark.sql.catalog.sylver.authentication.type', 'NONE')  # ✅ Move auth to "bronze" catalog
        .set('spark.sql.catalog.sylver.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')  # ✅ Use "bronze" prefix
        .set('spark.sql.catalog.sylver.s3.path-style-access', 'true')  # ✅ Configure S3 for "bronze"
        .set('spark.sql.catalog.sylver.s3.endpoint', AWS_S3_ENDPOINT)  # ✅ Fixed typo (no 's' at end)
        .set('spark.sql.catalog.sylver.warehouse', WAREHOUSE_GOLD)
        .set('spark.sql.catalog.sylver.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')  # ✅ Use "bronze" prefix
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY_ID)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.path.style.access','true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
)


In [3]:
## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
# print("Spark Running")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-10c0b760-6088-45f5-900f-0f4ea9468679;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.0 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.102.5 in central
	found software.amazon.awssdk#bundle;2.20.131 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.20.131 in central
	found software.amazon.awssdk#utils;2.20.131 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found softw

In [44]:
# spark.stop()

In [4]:
spark

In [27]:
print(spark.conf.get('spark.sql.catalog.nessie.s3.endpoint'))

http://minio:9000


In [244]:
## Create a Table
spark.sql("show databases in bronze;").show()

+---------------+
|      namespace|
+---------------+
|     DW_ny_bike|
|SylverDw_nybike|
+---------------+



In [29]:
## Create a Table
spark.sql(" CREATE DATABASE bronze.SylverDw_nybike;")

DataFrame[]

In [19]:
## Create a Table
spark.sql(" CREATE DATABASE IF NOT EXISTS sylver.DW_ny_bike;")

DataFrame[]

In [270]:
spark.sql("show table bronze.SylverDw_nybike;").show()

In [269]:
spark.sql("DROP TABLE bronze.SylverDw_nybike.trip_data_nybike;").show()

In [6]:
# spark.sql("show tables in nessie.dw_nybike;").show()
spark.sql("SHOW TBLPROPERTIES bronze.SylverDw_nybike.trip_data_nybike;").show()
# spark.sql("SELECT * FROM nessie.dw_nybike.trip_data_nybike_v2.snapshots;").show()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
| current-snapshot-id| 5479237530675047789|
|              format|     iceberg/parquet|
|      format-version|                   2|
|          gc.enabled|               false|
|    nessie.commit.id|5133ffd6f336e8366...|
|write.metadata.de...|               false|
|write.parquet.com...|                zstd|
+--------------------+--------------------+



In [57]:
spark.sql("""
CREATE TABLE bronze.DW_ny_bike.trip_data_nybike(
    dw_period_tag string,
    ride_id string,
	start_station_id string,
	start_station_name string,
	start_station_latitude string ,
	start_station_longitude string ,
	end_station_id string,
	end_station_name string,
	end_station_latitude string,
	end_station_longitude string,
	user_type string,
    gender string,
	customer_year_birth string,
    bike_id string,
	rideable_type string,
	start_at string,
	stop_at string, 
	trip_duration string
    )
USING iceberg
PARTITIONED BY (dw_period_tag)
;
""")

DataFrame[]

In [255]:
spark.sql("""
CREATE TABLE bronze.SylverDw_nybike.trip_data_nybike(
    trip_uuid string,
    dw_period_tag string,
	start_station_id string,
	start_station_name string,
	start_station_latitude string,
	start_station_longitude string,
	end_station_id string,
	end_station_name string,
	end_station_latitude string,
	end_station_longitude string,
	bike_id string,
	user_type string,
    enr_user_type string,
    enr_gender string,
	customer_year_birth  string,
	rideable_type string,
	start_at timestamp,
	stop_at timestamp,
	trip_duration double,
    year integer,
    quarter integer,
    quarter_name string,
    month integer,
    month_name string,
    day integer,
    weekday integer,
    weekday_name string
    )
USING iceberg
PARTITIONED BY (dw_period_tag)
    ;
    """)



DataFrame[]

In [31]:
# spark.sql("SHOW tables in bronze.DW_ny_bike;").show()
# spark.sql("ALTER TABLE bronze.DW_ny_bike.trip_data_nybike ALTER COLUMN start_at TYPE string;")
# spark.sql("DROP TABLE bronze.DW_ny_bike.trip_data_nybike;")
spark.sql("TRUNCATE TABLE bronze.SylverDw_nybike.trip_data_nybike;")

DataFrame[]

In [43]:
# spark.sql("select count(*) from bronze.DW_ny_bike.trip_data_nybike  limit 19;").show()

spark.sql("""
select 
    *
    --count(*) 
    --distinct(enr_user_type)
from bronze.SylverDw_nybike.trip_data_nybike 
where 
    dw_period_tag = '2015'
    limit 10
    ;"""
         ).show()

+---------+-------------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+-------+---------+-------------+----------+-------------------+-------------+--------+-------+-------------+----+-------+------------+-----+----------+---+-------+------------+
|trip_uuid|dw_period_tag|start_station_id|start_station_name|start_station_latitude|start_station_longitude|end_station_id|end_station_name|end_station_latitude|end_station_longitude|bike_id|user_type|enr_user_type|enr_gender|customer_year_birth|rideable_type|start_at|stop_at|trip_duration|year|quarter|quarter_name|month|month_name|day|weekday|weekday_name|
+---------+-------------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+-------+---------+-------------+----------+-------------------+-------------+--------+-------+---

In [27]:
# spark.sql("select distinct(dw_period_tag) from  bronze.DW_ny_bike.trip_data_nybike limit 19;").show()
spark.sql("""
select 
--count(*)
*
from  bronze.DW_ny_bike.trip_data_nybike 
where dw_period_tag like '200408%' 
limit 19;
""").show()
# spark.sql("DELETE FROM bronze.DW_ny_bike.trip_data_nybike WHERE dw_period_tag='2022';")

+-------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+---------+------+-------------------+-------+-------------+--------------------+--------------------+-------------+
|dw_period_tag|         ride_id|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|user_type|gender|customer_year_birth|bike_id|rideable_type|            start_at|             stop_at|trip_duration|
+-------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+---------+------+-------------------+-------+-------------+--------------------+--------------------+-------------+
|       200408|DDF3FD436F128E4E|         8149.09|Ogden Ave & W 1

In [4]:
## Insert Some Data
spark.sql("INSERT INTO nessie.names VALUES ('Alex Merced'), ('Dipankar Mazumdar'), ('Jason Hughes')").show()

++
||
++
++



+-----------------+
|             name|
+-----------------+
|      Alex Merced|
|Dipankar Mazumdar|
|     Jason Hughes|
+-----------------+



In [68]:
## Query the Data
spark.sql("SELECT * FROM nessie.dw_nybike.trip_data_nybike limit 3;").show()

+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+--------+-------+-------------+
|dw_period_tag|ride_id|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude| user_type|gender|customer_year_birth|bike_id|rideable_type|start_at|stop_at|trip_duration|
+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+--------+-------+-------------+
|         2014|   NULL|             479|     9 Ave & W 45 St|           40.76019252|            -73.9912551|           540|Lexington Ave & E...|         40.7414728

In [5]:
from pyspark.sql import DataFrame 
from typing import Optional
from pyspark.sql.functions import lit,concat,to_timestamp,unix_timestamp , from_unixtime ,try_to_timestamp,col,month , udf,isnull ,when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType , TimestampType , DoubleType , FloatType, DateType,NullType
from pyspark.sql import functions as f


bronze_schema_ny_bike = StructType([
    StructField("dw_period_tag", StringType(), nullable=True),
    StructField("ride_id", StringType(), nullable=True),
    StructField("start_station_id", StringType(), nullable=True),
    StructField("start_station_name", StringType(), nullable=True),
    StructField("start_station_latitude", StringType(), nullable=True),
    StructField("start_station_longitude", StringType(), nullable=True),
    StructField("end_station_id", StringType(), nullable=True),
    StructField("end_station_name", StringType(), nullable=True),
    StructField("end_station_latitude", StringType(), nullable=True),
    StructField("end_station_longitude", StringType(), nullable=True),
    StructField("bike_id", StringType(), nullable=True),
    StructField("user_type", StringType(), nullable=True),
    StructField("gender", StringType(), nullable=True),
    StructField("customer_year_birth", StringType(), nullable=True),
    StructField("rideable_type", StringType(), nullable=True),
    StructField("start_at", StringType(), nullable=True),
    StructField("stop_at", StringType(), nullable=True),
    StructField("trip_duration", StringType(), nullable=True)
])

sylver_schema_ny_bike = StructType([
    StructField("trip_uuid", StringType(), nullable=True),
    StructField("dw_period_tag", StringType(), nullable=True),
    StructField("start_station_id", StringType(), nullable=True),
    StructField("start_station_name", StringType(), nullable=True),
    StructField("start_station_latitude", StringType(), nullable=True),
    StructField("start_station_longitude", StringType(), nullable=True),
    StructField("end_station_id", StringType(), nullable=True),
    StructField("end_station_name", StringType(), nullable=True),
    StructField("end_station_latitude", StringType(), nullable=True),
    StructField("end_station_longitude", StringType(), nullable=True),
    StructField("bike_id", StringType(), nullable=True),
    StructField("enr_gender", StringType(), nullable=True),
    StructField("customer_year_birth", StringType(), nullable=True),
    StructField("rideable_type", StringType(), nullable=True),
    StructField("start_at", TimestampType(), nullable=True),
    StructField("stop_at", TimestampType(), nullable=True),
    StructField("trip_duration", DoubleType(), nullable=True),
    StructField("user_type", StringType(), nullable=True),
    StructField("enr_user_type", StringType(), nullable=True),
    StructField("year", IntegerType(), nullable=True),
    StructField("quarter", IntegerType(), nullable=True),
    StructField("quarter_name", StringType(), nullable=True),
    StructField("month", IntegerType(), nullable=True),
    StructField("month_name", StringType(), nullable=True),
    StructField("day", IntegerType(), nullable=True),
    StructField("weekday", IntegerType(), nullable=True),
    StructField("weekday_name", StringType(), nullable=True)
])

In [50]:
import yaml
def config_reader(path:str):
    # Load YAML config
    with open(path, "r") as f:
        config = yaml.safe_load(f)
    return config
    
def run(df,config):
    print("Cast To Datamodel initiated")
    data_model_from_df = list(dict(df.dtypes).keys())

    filtered_column = list(filter( lambda x: x not in data_model_from_df ,config['schema'].names ))
    if len(filtered_column) > 0 :
        for column in filtered_column:
            df =  df.withColumn(column, lit(None))

    df = df.select([
        col(field.name).cast(field.dataType).alias(field.name) 
        for field in config['schema'].fields
    ])
    return df

In [49]:
file_bike_2015='/opt/airflow/data/2015-citibike-tripdata/1_January/*.csv'
# file_bike_2017='/opt/airflow/data/2017-citibike-tripdata/*/*.csv'

In [54]:
!ls /opt/airflow/data/2015-citibike-tripdata/

10_October   12_December  2_February  4_April  6_June  8_August
11_November  1_January	  3_March     5_May    7_July  9_September


In [45]:
!head /opt/airflow/data/2015-citibike-tripdata/10_October/*.csv

==> /opt/airflow/data/2015-citibike-tripdata/10_October/201510-citibike-tripdata_1.csv <==
tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
171,10/1/2015 00:00:02,10/1/2015 00:02:54,388,W 26 St & 10 Ave,40.749717753,-74.002950346,494,W 26 St & 8 Ave,40.74734825,-73.99723551,24302,Subscriber,1973.0,1
593,10/1/2015 00:00:02,10/1/2015 00:09:55,518,E 39 St & 2 Ave,40.74780373,-73.9734419,438,St Marks Pl & 1 Ave,40.72779126,-73.98564945,19904,Subscriber,1990.0,1
233,10/1/2015 00:00:11,10/1/2015 00:04:05,447,8 Ave & W 52 St,40.76370739,-73.9851615,447,8 Ave & W 52 St,40.76370739,-73.9851615,17797,Subscriber,1984.0,1
250,10/1/2015 00:00:15,10/1/2015 00:04:25,336,Sullivan St & Washington Sq,40.73047747,-73.99906065,223,W 13 St & 7 Ave,40.73781509,-73.99994661,23966,Subscriber,1984.0,1
528,10/1/2015 00:00:17,10/1/2015 00:0

In [39]:
# dfinit =spark.read.format('csv').options(inferSchema='True', header='True',delimiter=',').load(file_bike_2015)
# df2017 =spark.read.format('csv').options(header='True').load(file_bike_2017)
dfinit= spark.sql("""SELECT * FROM bronze.DW_ny_bike.trip_data_nybike WHERE dw_period_tag='2015'; """)

In [40]:
dfinit.count()

9937969

In [8]:
dfinit_2= spark.sql("""SELECT * FROM bronze.DW_ny_bike.trip_data_nybike WHERE dw_period_tag='2015'; """)

In [ ]:
spark.sql("""SELECT * FROM bronze.DW_ny_bike.trip_data_nybike WHERE dw_period_tag='2015'; """)

In [31]:
config=config_reader('config.yaml')

In [9]:
dfinit_2.filter(dfinit_2.start_at.like('1/18/2015%')).show(3)

+-------------+-------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+--------------+--------------+-------------+
|dw_period_tag|ride_id|start_station_id|start_station_name|start_station_latitude|start_station_longitude|end_station_id|end_station_name|end_station_latitude|end_station_longitude| user_type|gender|customer_year_birth|bike_id|rideable_type|      start_at|       stop_at|trip_duration|
+-------------+-------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+--------------+--------------+-------------+
|         2015|   NULL|             499|Broadway & W 60 St|           40.76915505|           -73.98191841|           423| W 54 St & 9 Ave|    

In [72]:
dfinit_2.filter(dfinit_2.start_at.like('10/16/2015%')).show(3)

+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+-------------------+-------------------+-------------+
|dw_period_tag|ride_id|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude| user_type|gender|customer_year_birth|bike_id|rideable_type|           start_at|            stop_at|trip_duration|
+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+-------------------+-------------------+-------------+
|         2015|   NULL|             497|  E 17 St & Broadway|           40.73704984|          

In [135]:
config={'schema':sylver_schema_ny_bike}
df_casted = run(dfinit,config)

Cast To Datamodel initiated


In [138]:
df_casted.printSchema()

root
 |-- trip_uuid: string (nullable = true)
 |-- dw_period_tag: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: string (nullable = true)
 |-- start_station_longitude: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: string (nullable = true)
 |-- end_station_longitude: string (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- enr_gender: string (nullable = true)
 |-- customer_year_birth: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- start_at: timestamp (nullable = true)
 |-- stop_at: timestamp (nullable = true)
 |-- trip_duration: double (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- quarter_name: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- month_name: string (nul

In [130]:
# df_casted.select('*').when(isnull('start_at')).show(10)
# df_casted.filter(month(col('start_at')) ==  ).show(1)

In [12]:
df_renamed.printSchema()

root
 |-- trip_duration: integer (nullable = true)
 |-- start_at: string (nullable = true)
 |-- stop_at: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- customer_year_birth: string (nullable = true)
 |-- gender: integer (nullable = true)



In [69]:
# dfconverted = df.to(bronze_schema_ny_bike)
# df.select(df.start_at.cast("TIMESTAMP").alias('start_at')).show()
# df.select(to_timestamp(df.start_at, 'MM/dd/yyyy HH:mm:ss')).show(5)
# df.select(from_unixtime(unix_timestamp('start_at', 'MM/dd/yyyy HH:mm:ss')).cast(TimestampType()).alias("timestamp")).show(5)

from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime,to_timestamp,udf

# df_renamed_v2 = df_renamed.select(from_unixtime(unix_timestamp('start_at', 'MM/dd/yyyy hh:mm:ss ')).cast(TimestampType()).alias("timestamp"))
# change timestamp format

In [22]:
df_renamed_v2 = df_renamed.withColumn("timestamp", to_timestamp("start_at", "d/M/yyyy HH:mm:ss"))
# df = df.withColumn("timestamp", to_timestamp(df["date_string"], "M/d/yyyy HH:mm:ss"))


In [137]:
df_casted.printSchema()

root
 |-- trip_uuid: string (nullable = true)
 |-- dw_period_tag: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: string (nullable = true)
 |-- start_station_longitude: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: string (nullable = true)
 |-- end_station_longitude: string (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- enr_gender: string (nullable = true)
 |-- customer_year_birth: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- start_at: timestamp (nullable = true)
 |-- stop_at: timestamp (nullable = true)
 |-- trip_duration: double (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- quarter_name: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- month_name: string (nul

In [196]:
from datetime import datetime
inDate = "10/1/2014 00:00:27"
d = datetime.strptime(inDate, "%m/%d/%Y %H:%M:%S")
d
# strdate = d.strftime("%Y-%m-%d %H:%M:%S")

datetime.datetime(2014, 10, 1, 0, 0, 27)

In [198]:
str_guis=  d.strftime("%Y-%m-%d %H:%M:%S")

In [200]:
type(str_guis)

str

In [ ]:
datetime.datetime(2013, 4, 29, 15, 59, 2)
d.strftime("YYYYMMDD HH:mm:ss (%Y%m%d %H:%M:%S)")
'YYYYMMDD HH:mm:ss (20130429 15:59:02)'

In [203]:
# convert_udf = udf( lambda date: to_timestamp (datetime.strptime(date,'%m/%d/%Y %H:%M:%S'),TimestampType()))
convert_udf = udf( lambda date: to_timestamp (col((datetime.strptime(date,'%m/%d/%Y %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S"),TimestampType()))))

@udf
def convert_to_timestam(date_in):
    str_date = datetime.strptime(date_in,'%m/%d/%Y %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")
    # col_date = col(str_date)
    # return to_timestamp(col_date)
    return str_date

@udf
def convert_v2(date_in):
    date_datetime_format = datetime.strptime(date_in,'%m/%d/%Y %H:%M:%S') 
    str_date = date_datetime_format.strftime("%Y-%m-%d %H:%M:%S")
    return str_date

In [194]:
# dfinit.printSchema()
dfinit.show(2)

+------------+------------------+------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|         starttime|          stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+------------------+------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|        1027|10/1/2014 00:00:27|10/1/2014 00:17:34|             479|     9 Ave & W 45 St|           40.76019252|            -73.9912551|           540|Lexington Ave & E...|         40.74147286|         -73.98320928| 21376|Subscriber|    197

In [13]:
from pyspark.sql.functions import coalesce,lit,concat,to_timestamp,unix_timestamp , from_unixtime ,try_to_timestamp,col,month , udf,isnull ,when,cast,to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType , TimestampType , DoubleType , FloatType, DateType,NullType
from pyspark.sql import functions as f

In [170]:
row_1=dfinit.first()

In [187]:
col_start = row_1['starttime']

In [173]:
col_startime = convert_v2(row_1['starttime'])

In [10]:
dfinit_2.filter(dfinit_2.start_at.like('1/1/2015%')).show(4)

+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+----------------+-------------+-------------+
|dw_period_tag|ride_id|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude| user_type|gender|customer_year_birth|bike_id|rideable_type|        start_at|      stop_at|trip_duration|
+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+----------------+-------------+-------------+
|         2015|   NULL|             455|     1 Ave & E 44 St|           40.75001986|           -73.96905301|           26

In [15]:
# dfinit.select(convert_to_timestam("starttime").alias("start_time").cast(TimestampType())).printSchema()
# dfinit.withColumn('starttime',convert_to_timestam("starttime").cast(TimestampType())).printSchema()
# dfinit.withColumn('start_at',when(to_timestamp(dfinit.start_at).isNotNull(),to_timestamp(dfinit.start_at)).when(convert_v2(col("start_at")).isNotNull(),convert_v2(col("start_at")))).show(2)
# dfinit.withColumn('starttime',convert_v2('starttime')).withColumn('starttime',to_timestamp('starttime','yyyy-MM-dd HH:mm:ss')).printSchema()

# dfinit.withColumn('starttime',when(to_timestamp(dfinit.starttime).isNotNull(),to_timestamp(dfinit.starttime)).when(convert_v2(col("starttime")).isNotNull(),convert_v2(col("starttime")))).withColumn('starttime',to_timestamp('starttime')).show(1)


# dfinit.withColumn('starttime',to_timestamp(col('starttime'),'MM/d/yyyy HH:mm:ss')).filter(dfinit.starttime,isNull()).collect().show(1)
# dfinit.withColumn('start_at',df['']('starttime')).show(1)

# regex_pattern = r"^\d{1,2}/\d{1,2}/\d{4} \d{2}:\d{2}:\d{2}$"
# regex_pattern = r"^\d{1,2}/\d{1,2}/\d{4} \d{2}:\d{2}$"

regex_pattern = r"^\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2}$"
column='start_at'

dfinit_2=dfinit\
.filter(dfinit.start_at.like('1/%/2015%'))\
.withColumn(column,when(col(column).rlike(regex_pattern),concat(col(column),lit(':00'))).otherwise(col(column))) \


# .withColumn(
#     "enr_starttime",
#     coalesce(
#         # to_timestamp(col(column), 'M/d/yyyy HH:mm:ss')
#         to_timestamp(col(column), 'MM/d/yyyy H:mm:ss'),
#         to_timestamp(col(column), "yyyy-MM-dd HH:mm:ss"),
#         to_timestamp(col(column))
#     )
# ).show(1)

# dfinit.filter(dfinit.start_at.like('1/1/2015%')).withColumn("enr_starttime",when(col('start_at').rlike(regex_pattern),concat(col('start_at'),lit(':00'))).otherwise(col('start_at'))).show(3)

# dfinit.withColumn(
#     "enr_starttime",
#     when( col('starttime').rlike(regex_pattern),to_timestamp(col('starttime'))
#         col('starttime').rlike(regex_pattern)
# ).show(3)

# dfinit.withColumn('starttime',to_timestamp(col('starttime'),'MM/d/yyyy HH:mm:ss')).filter(dfinit.starttime.isNull()).collect()
# col("date_string").rlike(regex_pattern)

In [35]:
dfinit.filter(col('start_at').like('1/1/2015 __:__:__%')).show(7)

+-------------+-------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+---------+------+-------------------+-------+-------------+--------+-------+-------------+
|dw_period_tag|ride_id|start_station_id|start_station_name|start_station_latitude|start_station_longitude|end_station_id|end_station_name|end_station_latitude|end_station_longitude|user_type|gender|customer_year_birth|bike_id|rideable_type|start_at|stop_at|trip_duration|
+-------------+-------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+---------+------+-------------------+-------+-------------+--------+-------+-------------+
+-------------+-------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+---

In [45]:
regex_pattern = r"^\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2}$"
# .filter(dfinit.start_at.rlike('1/%/2015% __:__:__'))\

@udf
def safe_to_timestamp(column, fmt):
    columnv2 = None
    try:
        columnv2 = when(to_timestamp(col(column), fmt).isNotNull(), to_timestamp(col(column), fmt)).otherwise(None)
    except Exception as e:
        print(f"Error occurred: {e}")
    return columnv2

d='file_parquet' 
# .filter(col('start_at').like('1/%/2015 _:__:__'))\


dfinit\
.withColumn(column,when(col(column).rlike(regex_pattern),concat(col(column),lit(':00'))).otherwise(col(column))) \
.withColumn(
    "enr_starttime",
    coalesce(
        # safe_to_timestamp(col('start_at'), 'M/d/yyyy HH:mm:ss'),
        # safe_to_timestamp(col('start_at'),'M/d/yyyy H:mm:ss')
        to_timestamp(col(column), 'M/d/yyyy H:mm:ss'),
        # to_timestamp(col(column), 'MM/dd/yyyy H:mm:ss')
        # to_timestamp(col(column), 'MM/dd/yyyy H:mm:ss')
        # to_timestamp(col(column), "yyyy-MM-dd HH:mm:ss"),
        to_timestamp(col(column))
    )
).write.parquet(d, mode="overwrite")

In [ ]:
try:
    # Create DataFrame
    df = spark.createDataFrame(data, schema=schema)
    # Convert string dates to actual DateType
    df = df.withColumn("startDate", to_date(col("startDate"), "yyyy-MM-dd")) \
           .withColumn("endDate", to_date(col("endDate"), "yyyy-MM-dd"))
    # Show the DataFrame
    df.show()
except Exception as e:
    print(f"Error occurred: {e}")

In [46]:
dfinit_5=spark.read.parquet(d).show(100)

+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+-----------------+-----------------+-------------+-------------------+
|dw_period_tag|ride_id|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude| user_type|gender|customer_year_birth|bike_id|rideable_type|         start_at|          stop_at|trip_duration|      enr_starttime|
+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+-----------------+-----------------+-------------+-------------------+
|         2015|   NULL|             477|     W